In [1]:
from utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [2]:
board = OthelloBoardState()

In [ ]:
batch_size = 30
num_games = 30

def get_accuracy_from_boardstates(pred_board_state : Float[Tensor, "batch pos row col"], real_board_state : Float[Tensor, "batch pos row col"], not_flipped_real : Float[Tensor, "batch pos row col"]):
    return ((pred_board_state == real_board_state) & not_flipped_real & (real_board_state != EMPTY)).float().sum() / (not_flipped_real & (real_board_state != EMPTY)).float().sum()

for batch in tqdm(range(0, num_games, batch_size)):
    indeces = t.arange(batch, batch + batch_size)
    games = board_seqs_int[indeces].to(device)
    # logits = model(games[:, :-1])
    _, cache = model.run_with_cache(games[:, :-1])

    board_states = []
    for i in range(batch, batch+batch_size):
        moves = board_seqs_int[i]
        board = OthelloBoardState()
        board_states_game = []
        for move in moves:
            move = to_string(move.item())
            board.umpire(move)
            board_states_game += [board.state]
        board_states += [board_states_game]
    
    board_states = t.tensor(board_states).to(device)

    resid_post_6 = cache['resid_post', 6]
    linear_probe_6 = get_probe(6, "linear", "post")[0]
    flipped_probe_6 = get_probe(6, "flipped", "post")[0]
    real_board_state = einops.einsum(resid_post_6, linear_probe_6, 'b p d, d r c o -> b p r c o').argmax(-1)
    not_flipped_real = einops.einsum(resid_post_6, flipped_probe_6, 'b p d, d r c o -> b p r c o').argmax(-1) == NOT_FLIPPED

    for layer in range(8):
        resid_post = cache['resid_post', layer]
        attn_out = cache['attn_out', layer]
        linear_probe = get_probe(layer, "linear", "post")[0]
        pred_board_state_resid_post = einops.einsum(resid_post, linear_probe, 'b p d, d r c o -> b p r c o').argmax(-1)
        pred_board_state_attn_out = einops.einsum(attn_out, linear_probe, 'b p d, d r c o -> b p r c o').argmax(-1)
        accuracy_resid_post = get_accuracy_from_boardstates(pred_board_state_resid_post, real_board_state, not_flipped_real)
        accuracy_attn_out = get_accuracy_from_boardstates(pred_board_state_attn_out, real_board_state, not_flipped_real)
        print(f"Layer {layer} Resid Post Accuracy: {accuracy_resid_post.item()} Attn Out Accuracy: {accuracy_attn_out.item()}")

100%|██████████| 1/1 [00:00<00:00,  4.54it/s]

Layer 0 Resid Post Accuracy: 0.8166054487228394 Attn Out Accuracy: 0.7224854826927185
Layer 1 Resid Post Accuracy: 0.894166111946106 Attn Out Accuracy: 0.8383472561836243
Layer 2 Resid Post Accuracy: 0.940862238407135 Attn Out Accuracy: 0.8972365856170654
Layer 3 Resid Post Accuracy: 0.9653728008270264 Attn Out Accuracy: 0.9023836255073547
Layer 4 Resid Post Accuracy: 0.9817723631858826 Attn Out Accuracy: 0.9289707541465759
Layer 5 Resid Post Accuracy: 0.9924746751785278 Attn Out Accuracy: 0.8762224316596985
Layer 6 Resid Post Accuracy: 1.0 Attn Out Accuracy: 0.6971229910850525
Layer 7 Resid Post Accuracy: 0.9945334792137146 Attn Out Accuracy: 0.17141437530517578
